In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
import numpy as np

In [ ]:
#🛍️Quantity,💵sales & avg. value State-wise Analytics

# Page configuration
st.set_page_config(layout="wide")
st.title("🛍️Quantity,💵sales & avg. value State-wise Analytics")

# Load data
try:
    df = pd.read_csv("Amazon Sale Report.csv")
except FileNotFoundError:
    st.error("❌ File 'Amazon Sale Report.csv' not found in the current directory.")
    st.stop()

# Define numeric columns
numeric_cols = ['Quantity', 'avg. value', 'Sale_Amount']

# Preprocess
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
df['Shipping_State'] = df['Shipping_State'].astype(str)

# Sidebar - Filters
st.sidebar.header("🔍 Filters")

# Select up to 3 metrics
selected_metrics = st.sidebar.multiselect(
    "Select up to 3 Metrics",
    options=numeric_cols,
    default=['Sale_Amount'],
    max_selections=3
)

# State selection - searchable
all_states = sorted(df['Shipping_State'].dropna().unique())
selected_states = st.sidebar.multiselect(
    "Search & Select Shipping States",
    options=all_states,
    default=all_states[:5],
    help="You can search and select up to 37 states"
)

# Filter by selected states
filtered_df = df[df['Shipping_State'].isin(selected_states)]

# Range sliders for each metric
range_filters = {}
for metric in selected_metrics:
    col_values = pd.to_numeric(filtered_df[metric], errors='coerce').dropna()

    if col_values.empty or pd.isna(col_values.min()) or pd.isna(col_values.max()):
        st.warning(f"No valid numeric data for '{metric}' after filtering.")
        continue

    min_val = int(col_values.min())
    max_val = int(col_values.max())
    if min_val == max_val:
        max_val += 1
    step = max((max_val - min_val) // 100, 1)

    range_filters[metric] = st.sidebar.slider(
        f"{metric} Range",
        min_value=min_val,
        max_value=max_val,
        value=(min_val, max_val),
        step=step
    )

# Apply range filters
for metric in range_filters:
    min_val, max_val = range_filters[metric]
    filtered_df = filtered_df[(filtered_df[metric] >= min_val) & (filtered_df[metric] <= max_val)]

# Top N states input
top_n = st.sidebar.number_input(
    "Top N Shipping States per Metric", min_value=1, max_value=50, value=10
)

# Log scale option
use_log = st.sidebar.checkbox("Use logarithmic scale for Y-axis", value=False)

# Aggregation and plotting
if not filtered_df.empty and selected_metrics:
    grouped = filtered_df.groupby("Shipping_State")[selected_metrics].sum().reset_index()
    melted = pd.melt(grouped, id_vars="Shipping_State", value_vars=selected_metrics,
                     var_name="Metric", value_name="Value")

    # Filter out zero or negative values for log scale
    if use_log:
        melted = melted[melted["Value"] > 0]

    # Keep top N states per metric
    melted = melted.sort_values("Value", ascending=False).groupby("Metric").head(top_n)

    # Plot bar chart
    fig = px.bar(
        melted,
        x="Shipping_State",
        y="Value",
        color="Metric",
        barmode="group",
        title="Shipping State vs Selected Metrics",
        text_auto='.2s',
        log_y=use_log  # <- log scale toggle
    )
    fig.update_layout(
    xaxis_tickangle=-45,
    yaxis_type="log"
    )

    st.plotly_chart(fig, use_container_width=True)
else:
    st.info("No data to display. Please adjust your filters.")




In [ ]:
#🛍️Quantity,💵sales & avg. value City-wise Analytics

# Page configuration
st.set_page_config(layout="wide")
st.title("🛍️Quantity,💵sales & avg. value City-wise Analytics")
# Load your data
df = pd.read_csv("Amazon Sale Report.csv")

# Convert relevant columns to numeric (handle errors)
numeric_cols = ['Quantity', 'avg. value', 'Sale_Amount']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Sidebar - State filter
all_states = sorted(df['Shipping_State'].dropna().unique())
selected_states = st.sidebar.multiselect(
    "Select State(s)",
    options=all_states,
    default=all_states[:3]
)

# Filter dataframe by selected states
filtered_state_df = df[df['Shipping_State'].isin(selected_states)]

# Remove invalid/single-letter city names
filtered_state_df = filtered_state_df[filtered_state_df['Shipping_City'].str.len() > 1]

# Sidebar - City filter (cities only from selected states)
all_cities = sorted(filtered_state_df['Shipping_City'].dropna().unique())
selected_cities = st.sidebar.multiselect(
    "Select City(s)",
    options=all_cities,
    default=all_cities[:10]
)

# Filter dataframe by selected cities
filtered_city_df = filtered_state_df[filtered_state_df['Shipping_City'].isin(selected_cities)]

# Sidebar - Metrics multi-select (up to 3)
selected_metrics = st.sidebar.multiselect(
    "Select up to 3 Metrics",
    options=numeric_cols,
    default=['Quantity', 'Sale_Amount'],
    max_selections=3
)

# Sidebar - Y-axis range filter per selected metric
range_filters = {}
for metric in selected_metrics:
    col_vals = filtered_city_df[metric].dropna()
    if not col_vals.empty:
        min_val = float(col_vals.min())
        max_val = float(col_vals.max())
        range_filters[metric] = st.sidebar.slider(
            f"{metric} Range",
            min_value=min_val,
            max_value=max_val,
            value=(min_val, max_val)
        )

# Apply range filters
for metric in range_filters:
    min_val, max_val = range_filters[metric]
    filtered_city_df = filtered_city_df[
        (filtered_city_df[metric] >= min_val) & (filtered_city_df[metric] <= max_val)
    ]

# Sidebar - Top N cities filter
top_n = st.sidebar.number_input("Top N Cities", min_value=1, max_value=50, value=10)

# Aggregate and plot if data available
if not filtered_city_df.empty and selected_metrics:
    grouped = filtered_city_df.groupby("Shipping_City")[selected_metrics].sum().reset_index()

    # Sort descending by Quantity or first selected metric
    sort_metric = selected_metrics[0]
    grouped = grouped.sort_values(by=sort_metric, ascending=False).head(top_n)

    melted = pd.melt(grouped, id_vars="Shipping_City", value_vars=selected_metrics,
                     var_name="Metric", value_name="Value")

    fig = px.bar(
        melted,
        x="Shipping_City",
        y="Value",
        color="Metric",
        barmode="group",
        title=f"Top {top_n} Cities - Metrics vs Shipping City",
        text_auto='.2s'
    )
    fig.update_layout(xaxis_tickangle=-45)
    st.plotly_chart(fig, use_container_width=True)
else:
    st.info("No data to display. Please adjust your filters.")


In [ ]:
#🏙️ Promotion ID Count by Shipping State
import streamlit as st
import pandas as pd
import plotly.express as px

# Page setup
st.set_page_config(layout="wide")
st.title("📢 Promotion ID Count by Shipping State")

# Load data
df = pd.read_csv("Amazon Sale Report.csv")

# Preprocessing
df['Promotion_IDs'] = df['Promotion_IDs'].astype(str)
df['Shipping_State'] = df['Shipping_State'].astype(str)

# Sidebar - Filters
st.sidebar.header("🔍 Filters")

# State multi-select
all_states = sorted(df['Shipping_State'].dropna().unique())
selected_states = st.sidebar.multiselect(
    "Select Shipping States",
    options=all_states,
    default=all_states[:10]
)

# Filter by selected states
filtered_df = df[df['Shipping_State'].isin(selected_states)]

# Group by Shipping_State and count unique promotion IDs
grouped = filtered_df.groupby("Shipping_State")["Promotion_IDs"].nunique().reset_index(name="Promotion_Count")

# Range filter for Promotion_Count
if not grouped.empty:
    min_val = int(grouped["Promotion_Count"].min())
    max_val = int(grouped["Promotion_Count"].max())

    promo_min, promo_max = st.sidebar.slider(
        "Select Promotion ID Count Range (Y-axis)",
        min_value=min_val,
        max_value=max_val,
        value=(min_val, max_val)
    )

    # Apply Y-axis filter
    grouped = grouped[(grouped["Promotion_Count"] >= promo_min) & (grouped["Promotion_Count"] <= promo_max)]

    # Top N filter
    top_n = st.sidebar.number_input("Top N States by Promotion Count", 1, 50, 10)
    grouped = grouped.sort_values("Promotion_Count", ascending=False).head(top_n)

    # Plot
    if not grouped.empty:
        fig = px.bar(
            grouped,
            x="Shipping_State",
            y="Promotion_Count",
            title="🎯 Top States by Unique Promotion IDs",
            text_auto=True
        )
        fig.update_layout(xaxis_tickangle=-45)
        st.plotly_chart(fig, use_container_width=True)
    else:
        st.warning("No data available for the selected filters.")
else:
    st.warning("No data available for the selected filters.")


In [ ]:
#🏙️ Promotion ID Count by Shipping City
import streamlit as st
import pandas as pd
import plotly.express as px

# Page setup
st.set_page_config(layout="wide")
st.title("🏙️ Promotion ID Count by Shipping City")

# Load your CSV file
df = pd.read_csv("Amazon Sale Report.csv")

# Clean data
df['Shipping_City'] = df['Shipping_City'].astype(str)
df = df[df['Shipping_City'].str.len() > 1]             # Remove single-letter cities
df = df[df['Shipping_City'].str.lower() != 'nan']      # Remove NaNs in string format
df = df.dropna(subset=['Shipping_City', 'Shipping_State'])  # Drop actual NaNs
df['Promotion_ID_Count'] = pd.to_numeric(df['Promotion_ID_Count'], errors='coerce')
df = df.dropna(subset=['Promotion_ID_Count'])          # Drop rows where Promotion_ID_Count is NaN

# Sidebar Filters
st.sidebar.header("🔍 Filters")

# State filter
states = sorted(df['Shipping_State'].unique())
selected_states = st.sidebar.multiselect("Select State(s)", options=states, default=states)

# Filter cities based on selected states
filtered_df = df[df['Shipping_State'].isin(selected_states)]
available_cities = sorted(filtered_df['Shipping_City'].unique())
selected_cities = st.sidebar.multiselect("Select City(s)", options=available_cities, default=available_cities)

# Filter by cities
filtered_df = filtered_df[filtered_df['Shipping_City'].isin(selected_cities)]

# Y-axis min value
min_y = st.sidebar.number_input("Minimum Promotion ID Count (Y-axis filter)", min_value=0, value=5)

# Top N cities
top_n = st.sidebar.number_input("Top N Cities by Promotion ID Count", min_value=1, max_value=100, value=10)

# Group and filter
grouped = (
    filtered_df.groupby("Shipping_City")['Promotion_ID_Count']
    .sum()
    .reset_index()
)

# Filter by y-axis min value
grouped = grouped[grouped['Promotion_ID_Count'] >= min_y]

# Sort and limit to top N
grouped = grouped.sort_values(by="Promotion_ID_Count", ascending=False).head(top_n)

# Plotting
if not grouped.empty:
    fig = px.bar(
        grouped,
        x="Shipping_City",
        y="Promotion_ID_Count",
        title="Top Cities by Promotion ID Count",
        text_auto=True
    )
    fig.update_layout(xaxis_tickangle=-45)
    st.plotly_chart(fig, use_container_width=True)
else:
    st.warning("No data available for the selected filters.")


In [ ]:
#📦🚚order status / Shipping Service wrt to state and city
import streamlit as st
import pandas as pd
import plotly.express as px

st.set_page_config(layout="wide")
st.title("📦🚚order status / Shipping Service wrt to state and city")
# Load your data
df = pd.read_csv("Amazon Sale Report.csv")  # Replace with your actual file path

# Clean data
df = df.dropna(subset=["Shipping_City", "Shipping_State", "Order_Status", "Shipping_Service_Level"])
df = df[~df['Shipping_City'].str.fullmatch(r'^[A-Z]$', na=False)]  # Remove single-letter cities

st.title("Order Status / Shipping Service Level Analysis")

# Filter type
filter_type = st.radio("Select Filter Basis", ["State", "City"])

# Unique values
states = sorted(df['Shipping_State'].dropna().unique())
cities = sorted(df['Shipping_City'].dropna().unique())

# State filter
selected_states = st.multiselect("Select States", states)

# Dependent city filter
if selected_states:
    filtered_cities = sorted(df[df['Shipping_State'].isin(selected_states)]['Shipping_City'].dropna().unique())
else:
    filtered_cities = cities

selected_cities = st.multiselect("Select Cities", filtered_cities)

# Metric selection
metric_type = st.radio("Select Metric", ["Count", "Percentage"])
log_scale = st.checkbox("Use Log Scale")

# Value range filter
min_y, max_y = st.slider("Y-axis Value Range (Min/Max)", 0, 100000, (0, 100000), step=1000)

# Apply filters
if filter_type == "State":
    group_col = "Shipping_State"
    df = df[df['Shipping_State'].isin(selected_states)] if selected_states else df
elif filter_type == "City":
    group_col = "Shipping_City"
    df = df[df['Shipping_City'].isin(selected_cities)] if selected_cities else df

# Grouping
if metric_type == "Count":
    grouped = df.groupby([group_col, 'Order_Status']).size().reset_index(name='Count')
    grouped = grouped[(grouped['Count'] >= min_y) & (grouped['Count'] <= max_y)]
    y_col = 'Count'
elif metric_type == "Percentage":
    grouped = df.groupby([group_col, 'Order_Status']).size().reset_index(name='Count')
    grouped['Percentage'] = grouped.groupby(group_col)['Count'].transform(lambda x: x / x.sum() * 100)
    grouped = grouped[(grouped['Percentage'] >= min_y) & (grouped['Percentage'] <= max_y)]
    y_col = 'Percentage'

# Plot
fig = px.bar(
    grouped,
    x=group_col,
    y=y_col,
    color='Order_Status',
    title=f"{y_col} by {group_col} and Order Status",
    barmode='stack',
    text_auto='.2f' if metric_type == "Percentage" else True
)

if log_scale:
    fig.update_layout(yaxis_type='log')

fig.update_layout(xaxis_tickangle=-45)

# Display
st.plotly_chart(fig, use_container_width=True)


In [ ]:
#📦🚚 Order Status / Shipping Service Level by State or City (bar,pie,line graph)
import streamlit as st
import pandas as pd
import plotly.express as px

# Page setup
st.set_page_config(layout="wide")
st.title("📦 Order Status / Shipping Service Level by State or City")

# Load data
df = pd.read_csv("Amazon Sale Report.csv")

# Clean and filter data
df = df.dropna(subset=['Shipping_State', 'Shipping_City', 'Order_Status', 'Shipping_Service_Level'])
df['Shipping_City'] = df['Shipping_City'].astype(str)
df = df[df['Shipping_City'].str.len() > 1]

# Sidebar filters
st.sidebar.header("🔍 Filters")

# Select Y-axis dimension
y_col = st.sidebar.selectbox("Select Category (Y-axis)", ['Order_Status', 'Shipping_Service_Level'])

# Select X-axis dimension
x_col = st.sidebar.selectbox("Select Group By (X-axis)", ['Shipping_State', 'Shipping_City'])

# Select chart type
chart_type = st.sidebar.selectbox("Chart Type", ['Bar', 'Pie', 'Line'])

# Filter by state
states = sorted(df['Shipping_State'].unique())
selected_states = st.sidebar.multiselect("Select State(s)", states, default=states)

# Filter cities based on state selection
filtered_df = df[df['Shipping_State'].isin(selected_states)]
cities = sorted(filtered_df['Shipping_City'].unique())
selected_cities = st.sidebar.multiselect("Select City(s)", cities, default=cities)

# Filter data
df = df[df['Shipping_State'].isin(selected_states) & df['Shipping_City'].isin(selected_cities)]

# Group and count
grouped = df.groupby([x_col, y_col]).size().reset_index(name='Count')

# Y-axis value range
min_y = st.sidebar.number_input("Min Y-axis count", 0, 1000000, 0)
max_y = st.sidebar.number_input("Max Y-axis count (0 = no limit)", 0, 1000000, 0)

if max_y > 0:
    grouped = grouped[(grouped['Count'] >= min_y) & (grouped['Count'] <= max_y)]
else:
    grouped = grouped[grouped['Count'] >= min_y]

# Top N filter
top_n = st.sidebar.number_input(f"Top N {x_col}", min_value=1, max_value=100, value=10)
top_x = grouped.groupby(x_col)['Count'].sum().sort_values(ascending=False).head(top_n).index
grouped = grouped[grouped[x_col].isin(top_x)]

# Plotting
if grouped.empty:
    st.warning("No data available for selected filters.")
else:
    st.subheader(f"{chart_type} Chart of {y_col} by {x_col}")

    if chart_type == "Bar":
        fig = px.bar(
            grouped,
            x=x_col,
            y='Count',
            color=y_col,
            title=f"{y_col} by {x_col}",
            barmode='group',
            text_auto=True
        )
        fig.update_layout(xaxis_tickangle=-45)

    elif chart_type == "Line":
        fig = px.line(
            grouped,
            x=x_col,
            y='Count',
            color=y_col,
            title=f"{y_col} Trend across {x_col}",
            markers=True
        )

    elif chart_type == "Pie":
        pie_view = st.radio("Show pie chart by:", [x_col, y_col])
        pie_data = grouped.groupby(pie_view)['Count'].sum().reset_index()
        fig = px.pie(
            pie_data,
            names=pie_view,
            values='Count',
            title=f"{pie_view} Distribution of {y_col}",
            hole=0.3
        )

    st.plotly_chart(fig, use_container_width=True)


In [ ]:
#📊 B2B Distribution: State and City Analysis
import streamlit as st
import pandas as pd
import plotly.express as px

st.set_page_config(layout="wide")
st.title("📊 B2B Distribution: State and City Analysis")

# Load your CSV file
df = pd.read_csv("Amazon Sale Report.csv")

# Drop rows with missing critical data
df = df.dropna(subset=['Shipping_State', 'Shipping_City', 'Business_to_Business'])

# Ensure city names are valid
df = df[df['Shipping_City'].str.len() > 1]

# Sidebar selection
st.sidebar.header("🔍 Filters")
analysis_type = st.sidebar.radio("Select Analysis Type", ['B2B by State', 'B2B by City'])

if analysis_type == "B2B by State":
    # Group and calculate %
    state_b2b = df.groupby(['Shipping_State', 'Business_to_Business']).size().reset_index(name='Count')
    state_total = df.groupby('Shipping_State').size().reset_index(name='Total')
    state_b2b = state_b2b.merge(state_total, on='Shipping_State')
    state_b2b['Percent'] = (state_b2b['Count'] / state_b2b['Total']) * 100

    fig = px.bar(
        state_b2b,
        x='Shipping_State',
        y='Percent',
        color='Business_to_Business',
        barmode='stack',
        title="B2B % Distribution by State",
        text_auto='.2f'
    )
    fig.update_layout(xaxis_tickangle=-45)
    st.plotly_chart(fig, use_container_width=True)

else:
    # State filter
    states = sorted(df['Shipping_State'].unique())
    selected_states = st.sidebar.multiselect("Select State(s)", states, default=states)

    filtered_df = df[df['Shipping_State'].isin(selected_states)]

    # City filter
    cities = sorted(filtered_df['Shipping_City'].unique())
    selected_cities = st.sidebar.multiselect("Select City(s)", cities, default=cities)

    top_n = st.sidebar.number_input("Top N Cities", 1, 100, 10)

    # Apply filters
    df_city = filtered_df[filtered_df['Shipping_City'].isin(selected_cities)]
    top_cities = df_city['Shipping_City'].value_counts().head(top_n).index
    df_city = df_city[df_city['Shipping_City'].isin(top_cities)]

    if df_city.empty:
        st.warning("No data for selected filters.")
    else:
        # Group and calculate %
        city_b2b = df_city.groupby(['Shipping_City', 'Business_to_Business']).size().reset_index(name='Count')
        city_total = df_city.groupby('Shipping_City').size().reset_index(name='Total')
        city_b2b = city_b2b.merge(city_total, on='Shipping_City')
        city_b2b['Percent'] = (city_b2b['Count'] / city_b2b['Total']) * 100

        fig = px.bar(
            city_b2b,
            x='Shipping_City',
            y='Percent',
            color='Business_to_Business',
            barmode='stack',
            title="B2B % Distribution by City",
            text_auto='.2f'
        )
        fig.update_layout(xaxis_tickangle=-45)
        st.plotly_chart(fig, use_container_width=True)


In [ ]:
#👗👖👕 Product category / Size by State or City
import streamlit as st
import pandas as pd
import plotly.express as px

# Page setup
st.set_page_config(layout="wide")
st.title("👗👖👕 Product category / Size by State or City")


# Load data
df = pd.read_csv("Amazon Sale Report.csv")

# Data cleaning
df = df.dropna(subset=['Shipping_State', 'Product_Category', 'Product_Size', 'Quantity', 'Sale_Amount'])
df = df[~df['Shipping_State'].str.fullmatch(r"[A-Za-z]", na=False)]  # Remove single-letter states

# Sidebar filters
st.sidebar.header("Filters")

# Select metric
metric = st.sidebar.selectbox("Select Metric", ["Quantity", "Sale_Amount"])

# State filter
states = df['Shipping_State'].dropna().unique().tolist()
selected_states = st.sidebar.multiselect("Select States", sorted(states), default=states)

# Filter by selected states
df_filtered = df[df['Shipping_State'].isin(selected_states)]

# Y-axis range
min_val = int(df_filtered[metric].min())
max_val = int(df_filtered[metric].max())

lower, upper = st.sidebar.slider(f"{metric} Range", min_value=min_val, max_value=max_val, value=(min_val, max_val))
df_filtered = df_filtered[(df_filtered[metric] >= lower) & (df_filtered[metric] <= upper)]

# Top N states
top_n = st.sidebar.number_input("Select Top N States", min_value=1, max_value=50, value=10)

# Group by state and product category/size
grouped_cat = df_filtered.groupby(['Shipping_State', 'Product_Category'])[metric].sum().reset_index()
grouped_cat = grouped_cat.sort_values(by=metric, ascending=False).groupby('Shipping_State').head(top_n)

# Group by state and product size
grouped_size = df_filtered.groupby(['Shipping_State', 'Product_Size'])[metric].sum().reset_index()
grouped_size = grouped_size.sort_values(by=metric, ascending=False).groupby('Shipping_State').head(top_n)

# Bar Graph: Category vs State
st.subheader(f"{metric} by Product Category and State")
fig_cat = px.bar(grouped_cat, x="Shipping_State", y=metric, color="Product_Category", 
                 title=f"{metric} by Product Category and State", barmode="group")
st.plotly_chart(fig_cat, use_container_width=True)

# Bar Graph: Size vs State
st.subheader(f"{metric} by Product Size and State")
fig_size = px.bar(grouped_size, x="Shipping_State", y=metric, color="Product_Size", 
                  title=f"{metric} by Product Size and State", barmode="group")
st.plotly_chart(fig_size, use_container_width=True)

# Percentage Bar Graph: Product Category
st.subheader(f"Percentage Distribution of {metric} by Product Category per State")
df_cat_pct = grouped_cat.copy()
df_cat_pct["Percentage"] = df_cat_pct.groupby("Shipping_State")[metric].transform(lambda x: x / x.sum() * 100)
fig_pct_cat = px.bar(df_cat_pct, x="Shipping_State", y="Percentage", color="Product_Category", 
                     title=f"% {metric} by Product Category", barmode="stack")
st.plotly_chart(fig_pct_cat, use_container_width=True)

# Percentage Bar Graph: Product Size
st.subheader(f"Percentage Distribution of {metric} by Product Size per State")
df_size_pct = grouped_size.copy()
df_size_pct["Percentage"] = df_size_pct.groupby("Shipping_State")[metric].transform(lambda x: x / x.sum() * 100)
fig_pct_size = px.bar(df_size_pct, x="Shipping_State", y="Percentage", color="Product_Size", 
                      title=f"% {metric} by Product Size", barmode="stack")
st.plotly_chart(fig_pct_size, use_container_width=True)


In [ ]:
#👗👖👕Product Performance by City
import streamlit as st
import pandas as pd
import plotly.express as px

# Load data
df = pd.read_csv("Amazon Sale Report.csv")

st.set_page_config(layout="wide")
st.title("👗👖👕Product Performance by City")

# Clean data
df = df.dropna(subset=["Shipping_City", "Shipping_State", "Quantity", "Sale_Amount", "Product_Category", "Product_Size"])
df = df[df['Shipping_City'].str.len() > 1]  # Remove one-letter city names

# Sidebar Filters
st.sidebar.header("Filter Options")

# Metric selector
metric = st.sidebar.selectbox("Select Metric", ["Quantity", "Sale_Amount"])

# Choose Y-Axis category: Category or Size
y_axis_col = st.sidebar.selectbox("Select Y-Axis: Product Category or Size", ["Product_Category", "Product_Size"])

# Select state(s)
selected_states = st.sidebar.multiselect("Select State(s)", sorted(df["Shipping_State"].unique()))
filtered_df = df[df["Shipping_State"].isin(selected_states)] if selected_states else df

# Select cities based on selected states
available_cities = sorted(filtered_df["Shipping_City"].dropna().unique())
selected_cities = st.sidebar.multiselect("Select City(s)", available_cities)
if selected_cities:
    filtered_df = filtered_df[filtered_df["Shipping_City"].isin(selected_cities)]

# Y-axis range
min_val = int(filtered_df[metric].min())
max_val = int(filtered_df[metric].max())
y_min, y_max = st.sidebar.slider("Select Y-axis Range", min_val, max_val, (min_val, max_val))

# Top N cities
top_n = st.sidebar.number_input("Top N Cities", min_value=1, max_value=100, value=10)

# Filter by Y-axis range
filtered_df = filtered_df[(filtered_df[metric] >= y_min) & (filtered_df[metric] <= y_max)]

# Group data
grouped = filtered_df.groupby(["Shipping_City", y_axis_col])[metric].sum().reset_index()

# Get top N cities
top_cities = grouped.groupby("Shipping_City")[metric].sum().nlargest(top_n).index.tolist()
grouped = grouped[grouped["Shipping_City"].isin(top_cities)]

# ----- Bar Chart -----
st.subheader(f"{metric} by {y_axis_col} in Top {top_n} Cities")
fig = px.bar(grouped, x="Shipping_City", y=metric, color=y_axis_col,
             title=f"{metric} by {y_axis_col} in Cities",
             labels={"Shipping_City": "City", metric: f"Total {metric}"},
             height=600)
st.plotly_chart(fig, use_container_width=True)

# ----- Percentage Bar Chart -----
st.subheader(f"{metric} % Share by {y_axis_col} in Top {top_n} Cities")
grouped_pct = grouped.copy()
total_per_city = grouped_pct.groupby("Shipping_City")[metric].transform('sum')
grouped_pct["%"] = (grouped_pct[metric] / total_per_city) * 100

fig_pct = px.bar(grouped_pct, x="Shipping_City", y="%", color=y_axis_col,
                 title=f"Percentage Contribution of {y_axis_col} in {metric} by City",
                 labels={"Shipping_City": "City", "%": f"% of {metric}"},
                 height=600)
st.plotly_chart(fig_pct, use_container_width=True)


In [ ]:
#👗👖👕Product category Analysis by State
import streamlit as st
import pandas as pd
import plotly.express as px

# Title
st.set_page_config(layout="wide")
st.title("👗👖👕Product category Analysis by State")

# Load data
df = pd.read_csv("Amazon Sale Report.csv")

# Clean and filter data
df = df.dropna(subset=["Shipping_State", "Product_Category", "Product_Size"])
df = df[df["Shipping_State"].str.len() > 1]  # Remove one-letter states

# Sidebar filters
st.sidebar.header("Filters")
product_category = st.sidebar.selectbox("Select Product Category", df["Product_Category"].unique())
product_sizes = st.sidebar.multiselect("Select Product Sizes", df["Product_Size"].unique(), default=list(df["Product_Size"].unique()))
metric = st.sidebar.selectbox("Select Metric", ["Quantity", "Sale_Amount"])
selected_states = st.sidebar.multiselect("Select States", df["Shipping_State"].unique(), default=list(df["Shipping_State"].unique()))

# Filtered Data
df_filtered = df[(df["Product_Category"] == product_category) &
                 (df["Shipping_State"].isin(selected_states)) &
                 (df["Product_Size"].isin(product_sizes))]

# Aggregate data by state and product size
agg_df = df_filtered.groupby(["Shipping_State", "Product_Size"])[metric].sum().reset_index()
agg_df = agg_df[agg_df[metric] > 0]  # Remove zero entries

# Calculate total metric per state and merge for sorting
total_metric_by_state = agg_df.groupby("Shipping_State")[metric].sum().reset_index()
total_metric_by_state = total_metric_by_state.sort_values(by=metric, ascending=False)

# Merge sorted order into agg_df to ensure correct order
agg_df = agg_df.merge(total_metric_by_state, on="Shipping_State", suffixes=("", "_Total"))
agg_df = agg_df.sort_values(by=f"{metric}_Total", ascending=False)

# Stacked Bar Chart in descending order
fig1 = px.bar(
    agg_df,
    x="Shipping_State",
    y=metric,
    color="Product_Size",
    title=f"Stacked Bar Chart of {metric} by Product Size in Selected States",
    labels={"Shipping_State": "State", metric: metric, "Product_Size": "Size"},
    category_orders={"Shipping_State": agg_df["Shipping_State"].unique()}
)
st.plotly_chart(fig1, use_container_width=True)

# Treemap Chart
fig2 = px.treemap(
    agg_df,
    path=["Shipping_State", "Product_Size"],
    values=metric,
    title=f"Treemap of {metric} by Product Size in Selected States",
)
st.plotly_chart(fig2, use_container_width=True)


In [ ]:
###############################################################

In [ ]:
############  COLUMN ANALYSIS #####################

1. order staus cancelled - x axis and sum of quantuty n sales on y axis 

2. shipping service level - x axis and sum of quantity n sales on y axis 

3. order_status each category to put in broad 3-4 types and based on it
promotion id count + quantity + sales 

4. Product_Category - x axis and sum on y axis of quantity & Sale_Amount
5. Product_Size - x axis and sum on y axis 

6. keeping one constant ( Product_Category / Product_Size) 
and other as x axis and y axis has sum on y axis of quantity & Sale_Amount
#####################

import streamlit as st
import pandas as pd
import plotly.express as px

# Load CSV
df = pd.read_csv("Amazon Sale Report.csv")

# Clean data
df = df.dropna(subset=['Order_Status', 'Shipping_Service_Level', 'Product_Category', 'Product_Size'])
df = df[df['Shipping_City'].str.len() > 1]

# Sidebar: Metric selection
st.sidebar.title("Filters")

# Shared metric options
metrics_all = ['Quantity', 'Sale_Amount', 'avg. value']
metrics_qs = ['Quantity', 'Sale_Amount']

# Section 1 REMOVED

# Section 2: Shipping Service Level
st.subheader("1. Shipping Service Level")
metrics2 = st.multiselect("Select up to 2 metrics (Shipping Service Level)", metrics_qs, default=['Quantity'], max_selections=2)
if metrics2:
    service_df = df.groupby("Shipping_Service_Level")[metrics2].sum().reset_index().melt(id_vars="Shipping_Service_Level")
    fig2 = px.bar(service_df, x="Shipping_Service_Level", y="value", color="variable", barmode="group",
                  title="Shipping Service Level: Quantity & Sales")
    st.plotly_chart(fig2, use_container_width=True)

# Section 3: Order Status with PromoID/Qty/Sales
st.subheader("2. Order Status: Promo Count / Qty / Sales")
metrics3 = st.multiselect("Select up to 3 metrics", ['Promotion_ID_Count', 'Quantity', 'Sale_Amount'],
                          default=['Quantity'], max_selections=3)
if metrics3:
    status_df = df.groupby("Order_Status")[metrics3].sum().reset_index().melt(id_vars="Order_Status")
    fig3 = px.bar(status_df, x="Order_Status", y="value", color="variable", barmode="group",
                  title="Order Status: Promo Count / Quantity / Sales")
    st.plotly_chart(fig3, use_container_width=True)

# Section 4: Product Category
st.subheader("3. Product Category")
metrics4 = st.multiselect("Select up to 2 metrics (Product Category)", metrics_qs, default=['Quantity'], max_selections=2)
if metrics4:
    cat_df = df.groupby("Product_Category")[metrics4].sum().reset_index().melt(id_vars="Product_Category")
    fig4 = px.bar(cat_df, x="Product_Category", y="value", color="variable", barmode="group",
                  title="Product Category: Quantity & Sales")
    st.plotly_chart(fig4, use_container_width=True)

# Section 5: Product Size
st.subheader("4. Product Size")
metrics5 = st.multiselect("Select up to 2 metrics (Product Size)", metrics_qs, default=['Quantity'], max_selections=2)
if metrics5:
    size_df = df.groupby("Product_Size")[metrics5].sum().reset_index().melt(id_vars="Product_Size")
    fig5 = px.bar(size_df, x="Product_Size", y="value", color="variable", barmode="group",
                  title="Product Size: Quantity & Sales")
    st.plotly_chart(fig5, use_container_width=True)

# Section 6: Product Category vs Size (Improved)
# Section 5: Product Category vs Product Size (Improved & Paginated)
st.subheader("5. Product Category vs Product Size")

constant_col = st.selectbox("Group by", ["Product_Category", "Product_Size"])
dynamic_col = "Product_Size" if constant_col == "Product_Category" else "Product_Category"
metrics6 = st.multiselect("Select up to 2 metrics", metrics_qs, default=['Quantity'], max_selections=2)

if metrics6:
    grouped_df = df.groupby([constant_col, dynamic_col])[metrics6].sum().reset_index()
    options = grouped_df[constant_col].unique().tolist()
    selected_group = st.selectbox(f"Select {constant_col}", options)

    filtered_df = grouped_df[grouped_df[constant_col] == selected_group].melt(
        id_vars=[constant_col, dynamic_col], value_vars=metrics6)

    fig6 = px.bar(
        filtered_df,
        x=dynamic_col,
        y="value",
        color_discrete_sequence=["#1f77b4"],  # Monotonous blue
        barmode="group",
        title=f"{selected_group} - {dynamic_col}: Quantity & Sales",
        labels={dynamic_col: dynamic_col, "value": "Metric Value"},
        height=500
    )
    fig6.update_layout(xaxis_tickangle=-45)
    st.plotly_chart(fig6, use_container_width=True)


In [ ]:
################## TIME ANALYSIS ##################

In [ ]:
#📈 Monthly Trend of Quantity, Sales, and Average Value
import streamlit as st
import pandas as pd
import plotly.express as px

# Load CSV and preprocess
df = pd.read_csv("Amazon Sale Report.csv", low_memory=False)

# Convert Order_Date to datetime
df['Order_Date'] = pd.to_datetime(df['Order_Date'], errors='coerce')
df = df.dropna(subset=['Order_Date'])

# Convert numeric columns
df['Sale_Amount'] = pd.to_numeric(df['Sale_Amount'], errors='coerce')
df['avg. value'] = pd.to_numeric(df['avg. value'], errors='coerce')

# Extract month info
df['Month'] = df['Order_Date'].dt.month
df['MonthName'] = df['Order_Date'].dt.strftime('%B')  # Full month names
df['Year'] = df['Order_Date'].dt.year

# Create a MonthOrder to sort properly
month_order = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12
}
df['MonthOrder'] = df['MonthName'].map(month_order)

# Aggregate by MonthName and MonthOrder
monthly_agg = df.groupby(['MonthOrder', 'MonthName'])[['Quantity', 'Sale_Amount', 'avg. value']].sum().reset_index()
monthly_agg = monthly_agg.sort_values('MonthOrder')

# ----------------------
# 📈 LINE CHART
# ----------------------
st.subheader("📈 Monthly Trend of Quantity, Sales, and Average Value")

fig_line = px.line(
    monthly_agg,
    x='MonthName',
    y=['Quantity', 'Sale_Amount', 'avg. value'],
    title="Monthly Trends",
    markers=True,
    color_discrete_sequence=px.colors.qualitative.Set1,
    log_y=True
)

fig_line.update_layout(xaxis_title="Month", yaxis_title="Value", xaxis=dict(categoryorder='array', categoryarray=list(month_order.keys())))
st.plotly_chart(fig_line, use_container_width=True)

# ----------------------
# 🥧 PIE CHART
# ----------------------
st.subheader("🥧 Distribution by Month")

metric = st.selectbox("Select metric for pie chart", ['Quantity', 'Sale_Amount', 'avg. value'])

pie_df = monthly_agg[['MonthName', metric]]
fig_pie = px.pie(
    pie_df,
    names='MonthName',
    values=metric,
    title=f"Monthly Distribution of {metric}",
    color_discrete_sequence=px.colors.qualitative.Pastel1
)
st.plotly_chart(fig_pie, use_container_width=True)


In [ ]:
#Product Category / Size Analysis by Month

import streamlit as st
import pandas as pd
import plotly.express as px

# Load data
df = pd.read_csv("Amazon Sale Report.csv")

# Parse date and create month name and month number for sorting
df['Order_Date'] = pd.to_datetime(df['Order_Date'], errors='coerce', format='%m/%d/%Y')
df['Month'] = df['Order_Date'].dt.strftime('%B')  # e.g. January
df['MonthNum'] = df['Order_Date'].dt.month

# Drop rows with NaT in Order_Date after parsing (if any)
df = df.dropna(subset=['Order_Date'])

st.subheader("Product Category / Size Analysis by Month")

# Filters for month, dimension, and metric
months = sorted(df['Month'].unique(), key=lambda m: pd.to_datetime(m, format='%B').month)
selected_months = st.multiselect("Select Month(s)", options=months, default=months[:3])  # default first 3 months
dimension = st.selectbox("Select Dimension", ['Product_Category', 'Product_Size'])
metric = st.selectbox("Select Metric", ['Quantity', 'avg. value', 'Sale_Amount'])

# Filter data based on selected months
filtered_df = df[df['Month'].isin(selected_months)]

# 1. Continuous Time Line Graph
line_data = filtered_df.groupby(['MonthNum', 'Month', dimension])[metric].sum().reset_index()
line_data = line_data.sort_values('MonthNum')

fig_line = px.line(
    line_data,
    x='Month',
    y=metric,
    color=dimension,
    markers=True,
    title=f"{metric} over Months by {dimension}",
    color_discrete_sequence=px.colors.qualitative.Bold
)
if metric == 'Quantity':
    fig_line.update_yaxes(type='log', title=f"{metric} (log scale)")
else:
    fig_line.update_yaxes(title=metric)
st.plotly_chart(fig_line, use_container_width=True)

# 2. Percentage Stacked Bar Chart
bar_data = filtered_df.groupby([dimension, 'Month'])[metric].sum().reset_index()
bar_data['Percent'] = bar_data.groupby('Month')[metric].transform(lambda x: 100 * x / x.sum())

fig_bar = px.bar(
    bar_data,
    x='Month',
    y='Percent',
    color=dimension,
    title=f"Percentage Distribution of {metric} by {dimension} and Month",
    barmode='stack',
    color_discrete_sequence=px.colors.qualitative.Prism
)
fig_bar.update_yaxes(title='Percentage (%)')
st.plotly_chart(fig_bar, use_container_width=True)

# 3. Pie Chart: Filter by month & dimension (for each selected month)
st.markdown("### Pie Charts by Month")

for month in selected_months:
    pie_data = filtered_df[filtered_df['Month'] == month].groupby(dimension)[metric].sum().reset_index()
    fig_pie = px.pie(
        pie_data,
        names=dimension,
        values=metric,
        title=f"{metric} distribution in {month}",
        color_discrete_sequence=px.colors.qualitative.Set3
    )
    st.plotly_chart(fig_pie, use_container_width=True)
